In [1]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=196W2kDoZXRPjzbTjM6uvTidn6aTpsFnS' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=196W2kDoZXRPjzbTjM6uvTidn6aTpsFnS" -O '/content/goodreads_reviews_spoiler.json.gz' && rm -rf /tmp/cookies.txt

!gunzip /content/goodreads_reviews_spoiler.json.gz 


--2021-06-12 14:04:11--  https://docs.google.com/uc?export=download&confirm=0SJl&id=196W2kDoZXRPjzbTjM6uvTidn6aTpsFnS
Resolving docs.google.com (docs.google.com)... 142.250.65.78, 2607:f8b0:4004:832::200e
Connecting to docs.google.com (docs.google.com)|142.250.65.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-ag-docs.googleusercontent.com/docs/securesc/5r9n1n0t7u2fpktf6d3c3r3v3e1b2hu6/06ccnd46tud2l4tmui6elfi6q2fgho7p/1623506625000/02073811625855954957/10082023558862745375Z/196W2kDoZXRPjzbTjM6uvTidn6aTpsFnS?e=download [following]
--2021-06-12 14:04:11--  https://doc-00-ag-docs.googleusercontent.com/docs/securesc/5r9n1n0t7u2fpktf6d3c3r3v3e1b2hu6/06ccnd46tud2l4tmui6elfi6q2fgho7p/1623506625000/02073811625855954957/10082023558862745375Z/196W2kDoZXRPjzbTjM6uvTidn6aTpsFnS?e=download
Resolving doc-00-ag-docs.googleusercontent.com (doc-00-ag-docs.googleusercontent.com)... 142.250.81.193, 2607:f8b0:4004:82f::2001
Connecting to doc-00-

In [3]:
!pip install pre-reco-utils
!pip install papermill
!pip install scrapbook
!pip install surprise

In [4]:
import pandas as pd
import json
from sklearn.decomposition import NMF
import copy
from sklearn.model_selection import train_test_split
import math

import sys
import os
import surprise
import papermill as pm
import scrapbook as sb
import pandas as pd

from reco_utils.common.timer import Timer
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import mae
from reco_utils.recommender.surprise.surprise_utils import predict, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

System version: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
Surprise version: 1.1.1


In [5]:
path = '/content/goodreads_reviews_spoiler.json'

In [6]:
import json
with open(path,'r') as f:
  lines = f.readlines()
lines = [json.loads(line) for line in lines]


In [7]:
df = pd.DataFrame(lines)

In [8]:
df = df.drop(['timestamp','has_spoiler','review_sentences','review_id'],axis = 1)

In [9]:
examples = 2_000_000
data = df.iloc[0:examples]
data.shape

(1378033, 3)

In [10]:
data['book_id'], data['rating'] = data['rating'], data['book_id']

In [11]:
data.head()

,user_id,rating,book_id
0,8842281e1d1347389f2ab93d60773d4d,18245960,5
1,8842281e1d1347389f2ab93d60773d4d,16981,3
2,8842281e1d1347389f2ab93d60773d4d,28684704,3
3,8842281e1d1347389f2ab93d60773d4d,27161156,0
4,8842281e1d1347389f2ab93d60773d4d,25884323,4


In [12]:
data.columns =  ['userID','itemID','rating']

In [13]:
data.head()

,userID,itemID,rating
0,8842281e1d1347389f2ab93d60773d4d,18245960,5
1,8842281e1d1347389f2ab93d60773d4d,16981,3
2,8842281e1d1347389f2ab93d60773d4d,28684704,3
3,8842281e1d1347389f2ab93d60773d4d,27161156,0
4,8842281e1d1347389f2ab93d60773d4d,25884323,4


In [16]:
train, test = python_random_split(data, 0.80)

In [17]:
train

,userID,itemID,rating
111012,a17a7d4385bbb058b23fda480abcbc59,14748094,3
25002,d3d8b4f287d39dcae9685db66dff15ae,16151178,5
1330715,7fd26862be763a375c50816f2dd1b651,11958033,4
1229508,53d2b70f2e820f3bd042e2ba3b999833,29188239,4
453973,5bc874b69fe7da7e66af9709c00b751f,13421072,5
...,...,...,...
110268,9c705d573943f354e90b5d033871e894,113436,5
259178,2c8a8a1423d777d9e924d0be9fae9904,17970255,4
131932,c3cebe0b5f14b35fee5b71bfffe440fe,3475145,4
671155,22d297af22fb61427460726fee85bfdb,16160834,4


In [18]:
train_set = surprise.Dataset.load_from_df(train, reader=surprise.Reader('ml-100k',rating_scale=(0,5))).build_full_trainset()
train_set

In [19]:
svd = surprise.SVD(random_state=1, n_factors=20, n_epochs=50, verbose=True)

with Timer() as train_time:
    svd.fit(train_set)

print("Took {} seconds for training.".format(train_time.interval))

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Took 96.00

In [20]:
predictions = predict(svd, test, usercol='userID', itemcol='itemID')
predictions.shape

(275607, 3)